In [5]:

import sys

# setting path
sys.path.append('../smart_broker')

class Session():
    def __init__(self, http_method, ip, port):
        self.ip = ip
        self.port = port
        self.http_method = http_method

session = Session("http", "127.0.0.1", "8000")

# 1. Data Model Creation

In [6]:
# from smart_broker_client import data_model_client

# series_name = "bmi_mean"
# measurement_source_name = "Observation:Body Mass Index"
# type_aggregator = "AgregatorIntervalMean"
# unit = "kg/m2"

# data_model_client.create_series(session, series_name, measurement_source_name, type_aggregator, unit)

In [7]:
from smart_broker_client import data_model_client

data_model_id = data_model_client.create_date_frame(session, "data_frame_0")
data_model_client.data_frame_add_series(session, data_model_id, "bmi_mean", "Observation:Body Mass Index", "AgregatorIntervalMean")
data_model_client.data_frame_add_series(session, data_model_id, "bmi_first", "Observation:Body Mass Index", "AgregatorIntervalFirstOccurance")
data_model_client.data_frame_add_series(session, data_model_id, "bmi_last", "Observation:Body Mass Index", "AgregatorIntervalLastOccurance")
data_model_tabular_id = data_model_client.create_tabular_data(session)
data_model_tabular_id = data_model_client.tabular_add_dataframe(session, data_model_id, data_model_tabular_id)

In [8]:
data_model_id

'6bb4817a-e3cf-43bf-9b8f-2cd26a04a2ec'

# 2. Parse Data from Data Model Specification

In [9]:
from smart_broker_client import data_client

dataset_federation_id = "a892f738-4f6f-11ed-bdc3-0242ac120002"
dataset_federation_name = "r4sep2019_csvv1_20_1"

longitudinal_id = data_client.read_longitudinal_fhirv1(session)
tabular_dataset_id = data_client.parse_dataset_tabular_from_longitudinal(session, longitudinal_id, dataset_federation_id, dataset_federation_name, data_model_tabular_id)
data_frame_id = data_client.data_frame_tabular_select_data_frame(session, tabular_dataset_id, "data_frame_0")

In [10]:
from smart_broker_client import preprocessing_client

no_nan_data_frame_id = preprocessing_client.drop_na_data_frame(session, data_frame_id)

# 3. Grab series to work with in Statistics Lib

In [11]:
series_1_id = data_client.data_frame_select_series(session, no_nan_data_frame_id, "bmi_mean")
series_2_id = data_client.data_frame_select_series(session, no_nan_data_frame_id, "bmi_last")

print(series_1_id+" , "+series_2_id)

0e67a30d-e7a8-4724-bd0f-b2a7a572270a , 233e764d-0c64-4cac-b5dc-66cc6c8bb198


# 4. Run Statistics Library

In [12]:
from smart_broker_client import statistics_client

type_distribution="normalunit"
type_ranking="cdf"
alternative = "two-sided"
print(statistics_client.count(session,  series_1_id))
print(statistics_client.mean(session,  series_1_id))
print(statistics_client.chisquare(session,  series_1_id, series_2_id))
print(statistics_client.kolmogorovSmirnovTest(session,  series_1_id, type_distribution, type_ranking))
print(statistics_client.kurtosis(session,  series_1_id))
print(statistics_client.levene_test(session,  series_1_id, series_2_id))
print(statistics_client.mann_whitney_u_test(session,  series_1_id, series_2_id, alternative, type_ranking))
print(statistics_client.min_max(session,  series_1_id))
print(statistics_client.paired_t_test(session,  series_1_id, series_2_id, alternative))
print(statistics_client.pearson(session,  series_1_id, series_2_id, alternative))
print(statistics_client.skewness(session,  series_1_id))
print(statistics_client.spearman(session,  series_1_id, series_2_id, alternative, type_ranking))
print(statistics_client.student_t_test(session,  series_1_id, series_2_id, alternative))
print(statistics_client.variance(session,  series_1_id))
print(statistics_client.welch_t_test(session,  series_1_id, series_2_id, alternative))
print(statistics_client.wilcoxon_signed_rank_test(session,  series_1_id, series_2_id, alternative, type_ranking))

{'count': 20}
{'mean': 22.76433814224586}
{'chisquare': [380.0, 0.23583148842099294]}
{'kolmogorov_smirnov_test': [0.95, 1.9073486328125338e-26]}
{'kurtosis': -1.225866948308427}
{'f_statistic': 0.003760306204864892, 'p_value': 0.9514246952891092}
{'w_statistic': 180.0, 'p_value': 0.5978625674290079}
{'min': 11.76951154694196, 'max': 30.172100539128525}
{'t_statistic': -3.842534659324879, 'p_value': 0.00027445641167562454}
{'pearson': 0.9332515128886971, 'p_value': 1.9573960230445664e-09}
{'skewness': -0.08041628194553505}
{'spearman': 0.9505828978811472, 'p_value': 1.394346860195128e-10}
{'t_statistic': -0.999458438479299, 'p_value': 0.08097370675497725}
{'variance': 28.617281263722433}
{'t_statistic': -0.9994584384792992, 'p_value': 0.08097675209806848}
{'w_statistic': 0.9505828978811472, 'p_value': 1.394346860195128e-10}
